In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv


In [ ]:
wine_info = pd.read_csv("Datasets/winemag-data-130k-v2.csv")
wine_info.head()

In [ ]:
wine_info.columns

In [ ]:
reduced_wine_data_df = wine_info[["country", "points", "price", "variety", "province", "winery"]]

In [ ]:
drop_na_df = reduced_wine_data_df.dropna()

In [ ]:
country_group = drop_na_df.groupby("country")
province_group = drop_na_df.groupby("province")
winery_group = drop_na_df.groupby("winery")
country_by_province = drop_na_df.groupby(["country", "province"])
country_by_province.mean()